In [18]:
print(os.getcwd())

c:\Users\rody.vilchez\Documents\Capacitacion\Scripts\tests_automation\notebooks


In [22]:
#cargar hu desde hu.txt

import os
import sys

with open('../data/raw/UserHistory.txt', 'r') as f:
    hu = f.read()
# cargar cps.txt
with open('../data/raw/TestCases.txt', 'r') as f:
    cps = f.read()

# cargar expectResult.txt
with open('../data/raw/expectResults.txt', 'r') as f:
    expectResult = f.read()



In [23]:
cps = cps.split("\n")
cps = [x for x in cps if x != ""]

expectResult = expectResult.split("\n")
expectResult = [x for x in expectResult if x != ""]

In [21]:
Visma_Seek_Api = "sk-8811953733e4417abf2e0b41c56eb5a9" 

In [ ]:

from openai import OpenAI

def corregir_ortografia(cps):

    client = OpenAI(api_key=Visma_Seek_Api, base_url="https://api.deepseek.com")

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "Eres un experto en pruebas de software y tu tarea es corregir la ortografía de los casos de prueba."},
            {"role": "user", "content": "Este es la hu: " + hu},
            {"role": "user", "content": "Revisa los siguientes casos de prueba y corrige la ortografía (Solo ortografía, si es necesario alguna modificación menor) (NO ALTERES EL SENTIDO DEL CP)" + str(cps)},
            {"role": "user", "content": "Corrige la ortografía de los casos de prueba y devuelve el resultado en texto plano, sin formato. Al finalizar, devuelve las observaciones y el cp de original. Con el formato: OBS 1: xxxxxxx, CP: xxxxxxx. No incluyas comillas ni formato adicional."},
        ],
        stream=False
    )
    return response.choices[0].message.content

def corregir_ortografia(hu, cps):
    client = OpenAI(api_key=Visma_Seek_Api, base_url="https://api.deepseek.com")

    prompt = (
        f"Eres un experto en QA. Corrige solo errores ortográficos en los siguientes casos de prueba "
        f"relacionados con la historia de usuario (HU): {hu}.\n\n"
        "No cambies el significado ni el enfoque funcional del caso.\n"
        "Haz correcciones mínimas si son necesarias, solo ortografía o gramática leve.\n"
        "Formato de salida: texto plano.\n"
        "Al final de cada corrección, incluye una línea con el formato:\n"
        "OBS [n]: [descripción del error corregido], CP: [caso original].\n\n"
        f"Casos de prueba:\n{cps}"
    )

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "Eres un experto en pruebas de software."},
            {"role": "user", "content": prompt},
        ],
        stream=False
    )
    return response.choices[0].message.content


def corregir_expect_result(cps_with_expectResult):
    client = OpenAI(api_key=Visma_Seek_Api, base_url="https://api.deepseek.com")
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {
                "role": "system",
                "content": (
                    "Eres un experto en pruebas de software. "
                    "Tu tarea es corregir la ortografía y mejorar la redacción de los Expected Results, "
                    "manteniendo el sentido original y usando tiempo presente."
                )
            },
            {
                "role": "user",
                "content": (
                    "Aquí tienes los pares Caso de Prueba + Expected Result:\n"
                    f"{cps_with_expectResult}"
                )
            },
            {
                "role": "user",
                "content": (
                    "Devuelve únicamente los Expected Results corregidos en tiempo presente, mejora la redacción y corrige la ortografía, usa mayuscula inicial. "
                    "en texto plano y con este formato:\n"
                    "ExpRes1: <texto corregido 1>\n"
                    "ExpRes2: <texto corregido 2>\n"
                    "...\n\n"
                    "Al final, incluye un listado de observaciones de corrección con este formato:\n"
                    "OBS: <observación 1>\n"
                    "OBS: <observación 2>\n"
                    "No uses comillas ni ningún otro formato adicional."
                )
            }
        ],
        stream=False
    )
    return response.choices[0].message.content


# def formatear_cps(cps):
#     client = OpenAI(api_key=Visma_Seek_Api, base_url="https://api.deepseek.com")

#     response = client.chat.completions.create(
#         model="deepseek-chat",
#         messages=[
#             {"role": "system", "content": "Eres un experto en pruebas de software. Tu tarea es corregir la ortografía y mejorar la redacción de casos de prueba."},
#             {"role": "user", "content": (
#                 "Revisa los siguientes casos de prueba. Si encuentras valores entre paréntesis como (ALTO) o (ANCHO), "
#                 "expándelos con los valores correspondientes mencionados antes. Por ejemplo, si un caso incluye un tamaño 580x23 px "
#                 "y dice (Certificar un alto), reemplaza con 'Certificar un alto de 23 px'. Haz lo mismo para ancho, gap, margen, etc.\n\n"
#                 f"{cps}"
#             )},
#             {"role": "user", "content": "Devuelve solo el texto corregido y expandido, sin comillas ni formato adicional."}
#         ],
#         stream=False
#     )
#     return response.choices[0].message.content


In [27]:
# procesar en batches de 10

def procesar_en_batches(cps, batch_size=10):
    batches = [cps[i:i + batch_size] for i in range(0, len(cps), batch_size)]
    resultados = []
    for batch in batches:
        resultado = corregir_ortografia(hu, batch)
        resultados.extend(resultado.split("\n"))
    return resultados

def procesar_en_batches_expectResult(cps, expectResult, batch_size=10):
    cps_with_expectResult = []
    for i in range(len(cps)):
        cps_with_expectResult.append(cps[i] + "ExpRes:" + expectResult[i])
    batches = [cps_with_expectResult[i:i + batch_size] for i in range(0, len(cps), batch_size)]
    resultados = []
    for batch in batches:
        resultado = corregir_expect_result(batch)
        resultados.extend(resultado.split("\n"))
    return resultados

In [28]:

if cps.__len__() == len(expectResult):
    print("Los casos de prueba y los resultados esperados tienen la misma longitud")
    #resultados = procesar_en_batches_expectResult(cps, expectResult)

else:
    print(f"Casos de prueba {len(cps)} \nresultados esperados {len(expectResult)}")



#resultados = procesar_en_batches(cps)
resultados = procesar_en_batches_expectResult(cps, expectResult)
print(resultados)



Los casos de prueba y los resultados esperados tienen la misma longitud
['ExpRes1: En Social se muestra el botón Rechazar invitación en la pestaña Explorar grupos para grupos públicos del usuario BP.', 'ExpRes2: En Social, al hacer clic en el botón Rechazar invitación en Explorar grupos de un grupo público, el botón se muestra deshabilitado para el usuario BP.', 'ExpRes3: En Social, al hacer clic en el botón Rechazar invitación en Explorar grupos de un grupo público, el botón Unirse a grupo se muestra deshabilitado para el usuario BP.', 'ExpRes4: En Social, al hacer clic en el botón Rechazar invitación en Explorar grupos de un grupo público, se muestra un loader para el usuario BP.', 'ExpRes5: En Social, en Explorar grupos (público), al finalizar el loader generado al rechazar invitación, se muestra una alerta de éxito para el usuario BP.', 'ExpRes6: En Social, en Explorar grupos (público), al rechazar la invitación se muestra el texto Invitación rechazada en la alerta de éxito para el

In [29]:
print(resultados)

['ExpRes1: En Social se muestra el botón Rechazar invitación en la pestaña Explorar grupos para grupos públicos del usuario BP.', 'ExpRes2: En Social, al hacer clic en el botón Rechazar invitación en Explorar grupos de un grupo público, el botón se muestra deshabilitado para el usuario BP.', 'ExpRes3: En Social, al hacer clic en el botón Rechazar invitación en Explorar grupos de un grupo público, el botón Unirse a grupo se muestra deshabilitado para el usuario BP.', 'ExpRes4: En Social, al hacer clic en el botón Rechazar invitación en Explorar grupos de un grupo público, se muestra un loader para el usuario BP.', 'ExpRes5: En Social, en Explorar grupos (público), al finalizar el loader generado al rechazar invitación, se muestra una alerta de éxito para el usuario BP.', 'ExpRes6: En Social, en Explorar grupos (público), al rechazar la invitación se muestra el texto Invitación rechazada en la alerta de éxito para el usuario BP.', 'ExpRes7: En Social, en Explorar grupos (público), si el 

In [30]:
filter_r = [x.strip() for x in resultados if x != "" and (x.strip()[:3] == "SOC")] # esto filtra que empiece por RL, EM, CP o H
filter_obs = [x.strip() for x in resultados if x != "" and x.strip()[:6] == "ExpRes"]
filter_feedback = [x.strip() for x in resultados if x != "" and (x.strip()[:3] == "OBS")] # esto filtra que empiece por RL, EM, CP o H

In [31]:
filter_r

[]

In [32]:

print(f"resultados {filter_obs}")

if filter_obs.__len__() == cps.__len__():
    print("Todos los casos de prueba fueron corregidos exitosamente.")
else:
    print(f"resultados {filter_obs.__len__()} no son iguales a cps {cps.__len__()}")

resultados ['ExpRes1: En Social se muestra el botón Rechazar invitación en la pestaña Explorar grupos para grupos públicos del usuario BP.', 'ExpRes2: En Social, al hacer clic en el botón Rechazar invitación en Explorar grupos de un grupo público, el botón se muestra deshabilitado para el usuario BP.', 'ExpRes3: En Social, al hacer clic en el botón Rechazar invitación en Explorar grupos de un grupo público, el botón Unirse a grupo se muestra deshabilitado para el usuario BP.', 'ExpRes4: En Social, al hacer clic en el botón Rechazar invitación en Explorar grupos de un grupo público, se muestra un loader para el usuario BP.', 'ExpRes5: En Social, en Explorar grupos (público), al finalizar el loader generado al rechazar invitación, se muestra una alerta de éxito para el usuario BP.', 'ExpRes6: En Social, en Explorar grupos (público), al rechazar la invitación se muestra el texto Invitación rechazada en la alerta de éxito para el usuario BP.', 'ExpRes7: En Social, en Explorar grupos (públi

In [33]:
import re 
# limpiar expRes1: 
for r in range(len(filter_obs)):
    filter_obs[r] = re.sub(r"ExpRes\d+: ", "", filter_obs[r])


In [35]:
# concatenar los resultados
r = "\n".join(filter_r)
print(r)
# guardar los resultados en un archivo de texto en utf8
with open("../data/processed/resultados.txt", "w", encoding="utf-8") as f:
    f.write(r)

obs = "\n".join(filter_obs)
print(obs)
# guardar los resultados en un archivo de texto
with open("../data/processed/observaciones.txt", "w", encoding="utf-8") as f:
    f.write(obs)

feedback = "\n".join(filter_feedback)
with open("../data/processed/feedback.txt", "w", encoding="utf-8") as f:
    f.write(feedback)


En Social se muestra el botón Rechazar invitación en la pestaña Explorar grupos para grupos públicos del usuario BP.
En Social, al hacer clic en el botón Rechazar invitación en Explorar grupos de un grupo público, el botón se muestra deshabilitado para el usuario BP.
En Social, al hacer clic en el botón Rechazar invitación en Explorar grupos de un grupo público, el botón Unirse a grupo se muestra deshabilitado para el usuario BP.
En Social, al hacer clic en el botón Rechazar invitación en Explorar grupos de un grupo público, se muestra un loader para el usuario BP.
En Social, en Explorar grupos (público), al finalizar el loader generado al rechazar invitación, se muestra una alerta de éxito para el usuario BP.
En Social, en Explorar grupos (público), al rechazar la invitación se muestra el texto Invitación rechazada en la alerta de éxito para el usuario BP.
En Social, en Explorar grupos (público), si el rechazo es exitoso, la card del grupo rechazado desaparece sin actualizar la págin

## Criterio experto

- [] Setear temperatura
- [x]
